Look at sample lifetimes and under stand some numbers that we might expect if everything works 'well'.

In [1]:
import sys, os
sys.path.append(os.pardir)

from hep_tables import xaod_table, make_local
from func_adl import EventDataset

from data import get_ds

In [2]:
ds = get_ds(mH=125, mS=35, lifetime=5, campaign='mc16e')
assert len(ds) == 1
ds

,mH,mS,Lifetime,MCCampaign,RucioDSName,Use,Comments
18,125,35,5,mc16e,mc16_13TeV.311312.MadGraphPythia8EvtGen_A14NNP...,1,NaN


In [3]:
dataset = EventDataset(f'localds://mc16_13TeV:{ds["RucioDSName"].values[0]}')
df = xaod_table(dataset)

In [4]:
truth = df.TruthParticles('TruthParticles')
llp_truth = truth[truth.pdgId == 35]

In [5]:
n_llp = make_local(llp_truth.Count())

In [6]:
n_llp

array([2, 2, 2, ..., 2, 2, 2])

In [ ]:
lx_prod = make_local(llp_truth.prodVtx.x)
lx_decay = make_local(llp_truth.decayVtx.x, force_rerun=True)
lx = lx_prod-lx_decay

In [ ]:
lx